# KAGGLE COMPETITION RECVIS TP3
Paul CHAUVIN

paulchauvin97@gmail.com

##Environment


In [13]:
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tqdm import tqdm
import PIL.Image as Image
import zipfile
import torchvision
import pandas as pd
import json
!pip install chainercv
import chainercv
from chainercv.datasets import voc_bbox_label_names
from chainercv.links import FasterRCNNVGG16, SSD300, SSD512, YOLOv3, YOLOv2
from chainercv.utils import read_image
from chainercv.visualizations import vis_bbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
use_cuda = torch.cuda.is_available()
import sys
sys.argv=['']
del sys

##Crop images

In [16]:
path = '/content/drive/MyDrive/Cours_MVA/object_recognition/Kaggle_TP3/bird_dataset_cropped'
model4 = FasterRCNNVGG16(pretrained_model='voc07')
model0 = SSD300(pretrained_model='voc0712')
model2 = SSD512(pretrained_model='voc0712')
model6 = YOLOv3(pretrained_model='voc0712')
model7 = YOLOv2(pretrained_model='voc0712')

In [ ]:
def save_img(bboxes,full_path):
    img = Image.open(full_path)
    top, left , bottom, right  = bboxes[0][0]
    cropped = img.crop( ( left, top, right, bottom) )  # size: 45, 45
    try:
        cropped.save(full_path, "JPEG", quality=80, optimize=True, progressive=True)
    except IOError:
        PIL.ImageFile.MAXBLOCK = cropped.size[0] * cropped.size[1]
        cropped.save(full_path, "JPEG", quality=80, optimize=True, progressive=True)
def comparer_model(model, img):
    bboxes0, labels0, scores0 = model.predict([img])
    if len(bboxes0[0])>0:
        if labels0[0][0]==2:
            maximum_proba=scores0[0][0]
            return maximum_proba, bboxes0
    return(-1,-1)
def main_crop(path):
    for dossier, sous_dossiers, fichiers in os.walk(path):
        print (sous_dossiers)
        for num, fichier in enumerate(fichiers):
            if num%50==0:
                print(num)
            full_path = os.path.join(dossier, fichier)
            img = read_image(full_path)
            maximum0, bboxes_f0 = comparer_model(model0, img)
            maximum1, bboxes_f2 = comparer_model(model2, img)
            maximum2, bboxes_f4 = comparer_model(model4, img)
            maximum3, bboxes_f6 = comparer_model(model6, img)
            maximum4, bboxes_f7 = comparer_model(model7, img)
            
            if max([maximum0 ,maximum1, maximum2, maximum3, maximum4])<=0.9:
                continue
            if maximum0==max([maximum0 ,maximum1, maximum2, maximum3, maximum4]):
                save_img(bboxes_f0,full_path)
                continue
            if maximum1==max([maximum1, maximum2, maximum3, maximum4]):
                save_img(bboxes_f2,full_path)
                continue
            if maximum2==max([maximum2, maximum3, maximum4]):
                save_img(bboxes_f4,full_path)
                continue
            if maximum3==max([maximum3, maximum4]):
                save_img(bboxes_f6,full_path)
                continue
            if maximum4>max([maximum1, maximum2, maximum3, maximum4]):
                save_img(bboxes_f7,full_path)
                continue
main_crop("./bird_dataset_cropped/test_images")
main_crop("./bird_dataset_cropped/train_images")
main_crop("./bird_dataset_cropped/val_images")

def plot(full_path,model):
    # Read an RGB image and return it in CHW format.
    img = read_image(full_path)
    bboxes, labels, scores = model.predict([img])
    print(labels, scores)
    vis_bbox(img, bboxes[0], labels[0], scores[0],
             label_names=voc_bbox_label_names)
    plt.show()


['mistery_category']
[]
0
50
100
150
200


In [ ]:
full_path1 = '//content/drive/MyDrive/Cours MVA/Object recognition/Kaggle_TP3/bird_dataset/test_images/mistery_category/0a4a063484eba8522de89bdb24878e74.jpg'
full_path2 = '//content/drive/MyDrive/Cours MVA/Object recognition/Kaggle_TP3/bird_dataset/test_images/mistery_category/0b7b64c3eeae20e8f71923914a8643ef.jpg'
full_path3 = '//content/drive/MyDrive/Cours MVA/Object recognition/Kaggle_TP3/bird_dataset/train_images/004.Groove_billed_Ani/Groove_Billed_Ani_0002_1670.jpg'
full_path4 = '//content/drive/MyDrive/Cours MVA/Object recognition/Kaggle_TP3/bird_dataset/train_images/009.Brewer_Blackbird/Brewer_Blackbird_0004_2345.jpg'


In [ ]:
plot(full_path1,model0)
plot(full_path1,model2)
plot(full_path1,model4)
plot(full_path1,model6)
plot(full_path1,model7)

In [ ]:
plot(full_path2,model0)
plot(full_path2,model2)
plot(full_path2,model4)
plot(full_path2,model6)
plot(full_path2,model7)

In [ ]:
plot(full_path3,model0)
plot(full_path3,model2)
plot(full_path3,model4)
plot(full_path3,model6)
plot(full_path3,model7)

In [ ]:
plot(full_path4,model0)
plot(full_path4,model2)
plot(full_path4,model4)
plot(full_path4,model6)
plot(full_path4,model7)

##Git Clone

In [ ]:
!pip install --upgrade tf_slim
!pip install nets
!pip install preprocessing
%cd /content/drive/MyDrive/Cours_MVA/object_recognition/Kaggle_TP3
!git clone --recursive https://github.com/richardaecn/cvpr18-inaturalist-transfer.git

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import os
import sys
import time
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tf_slim as slim
sys.path.insert(0, './slim/nets/')
#import nets
import preprocessing
from slim.nets import inception_resnet_v2
from slim.nets import inception
from slim.preprocessing import inception_preprocessing

data_dir = '/content/drive/MyDrive/Cours_MVA/object_recognition/Kaggle_TP3/cvpr18_inaturalist_transfer/data'
# dataset needs to be one of ['ILSVRC2012', 'inat2017', 'cub_200', 'flower_102',
# 'stanford_cars', 'stanford_dogs', 'aircraft', 'nabirds', 'food_101']
dataset = 'cub_200'
# base_network needs to be one of ['InceptionV3', 'InceptionV3SE',
# 'InceptionV4', 'InceptionResnetV2', 'InceptionResnetV2SE', 'ResNet50',
# 'ResNet101', 'ResNet152']
base_network = 'InceptionV3'
checkpoints_path ='/content/drive/MyDrive/Cours_MVA/object_recognition/Kaggle_TP3/cvpr18_inaturalist_transfer/inception_v3_iNat_299.ckpt'# './checkpoints/inception/inception_v3_iNat_299.ckpt'
# base_network = 'ResNet101'
# checkpoints_path = './checkpoints/resnet/resnet_101_ILSVRC_iNat_299.ckpt'

image_size = 299
moving_average_decay = 0.9999
fea_dim = 2048

# Read train and val list.
train_list = []
val_list = []
for line in open(os.path.join(data_dir, dataset, 'train.txt'), 'r'):
    train_list.append(
        (os.path.join(data_dir, dataset, line.strip().split(': ')[0]),
        int(line.strip().split(': ')[1])))
for line in open(os.path.join(data_dir, dataset, 'val.txt'), 'r'):
    val_list.append(
        (os.path.join(data_dir, dataset, line.strip().split(': ')[0]),
        int(line.strip().split(': ')[1])))
print('Length of train: %d' %len(train_list))
print('Length of val: %d' %len(val_list))

# Base network architecture
if base_network == 'InceptionV3':
    endpoint = 'Mixed_7c'
    arg_scope = inception.inception_v3_arg_scope()
elif base_network == 'InceptionV3SE':
    endpoint = 'Mixed_7c'
    arg_scope = inception.inception_v3_se_arg_scope()
elif base_network == 'InceptionV4':
    endpoint = 'Mixed_7d'
    arg_scope = inception.inception_v4_arg_scope()
elif base_network == 'InceptionResnetV2':
    endpoint = 'Conv2d_7b_1x1'
    arg_scope = inception.inception_resnet_v2_arg_scope()
elif base_network == 'InceptionResnetV2SE':
    endpoint = 'Conv2d_7b_1x1'
    arg_scope = inception.inception_resnet_v2_se_arg_scope()
elif base_network[:6] == 'ResNet':
    layers = base_network.split('ResNet')[1]
    base_network = 'ResNet'

# Feature extraction.
fea_train = np.zeros((len(train_list), fea_dim), dtype=np.float32)
label_train = np.zeros((len(train_list), ), dtype=np.int32)
fea_val = np.zeros((len(val_list), fea_dim), dtype=np.float32)
label_val = np.zeros((len(val_list), ), dtype=np.int32)

with tf.Graph().as_default():
    tf_global_step = tf.train.get_or_create_global_step()
    image_path = tf.placeholder(tf.string)
    image = tf.image.decode_jpeg(tf.read_file(image_path), channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = inception_preprocessing.preprocess_image(image,
                                                     image_size,
                                                     image_size,
                                                     is_training=False)
    images  = tf.expand_dims(image, 0)

    if base_network == 'ResNet':
        with slim.arg_scope(resnet_v2.resnet_arg_scope(use_batch_norm=True)):
            if layers == '50':
                net, _ = resnet_v2.resnet_v2_50(images, is_training=False)
            elif layers == '101':
                net, _ = resnet_v2.resnet_v2_101(images, is_training=False)
            elif layers == '152':
                net, _ = resnet_v2.resnet_v2_152(images, is_training=False)
    else:
        with slim.arg_scope(arg_scope):
            slim_args = [slim.batch_norm, slim.dropout]
            with slim.arg_scope(slim_args, is_training=False):
                with tf.variable_scope(base_network, reuse=None) as scope:
                    if base_network == 'InceptionV3':
                        net, _ = inception.inception_v3_base(
                            images, final_endpoint=endpoint, scope=scope)
                    elif base_network == 'InceptionV3SE':
                        net, _ = inception.inception_v3_se_base(
                            images, final_endpoint=endpoint, scope=scope)
                    elif base_network == 'InceptionV4':
                        net, _ = inception.inception_v4_base(
                            images, final_endpoint=endpoint, scope=scope)
                    elif base_network == 'InceptionResnetV2':
                        net, _ = inception.inception_resnet_v2_base(
                            images, final_endpoint=endpoint, scope=scope)
                    elif base_network == 'InceptionResnetV2SE':
                        net, _ = inception.inception_resnet_v2_se_base(
                            images, final_endpoint=endpoint, scope=scope)
    net = tf.reduce_mean(net, [0,1,2])

    variable_averages = tf.train.ExponentialMovingAverage(
        moving_average_decay, tf_global_step)
    variables_to_restore = variable_averages.variables_to_restore()
    init_fn = slim.assign_from_checkpoint_fn(
        checkpoints_path, variables_to_restore)

    config_sess = tf.ConfigProto(allow_soft_placement=True)
    config_sess.gpu_options.allow_growth = True
    with tf.Session(config=config_sess) as sess:
        init_fn(sess)
        start = time.time()
        print('Feature extraction on training set...')
        for i in range(len(fea_train)):
            if i%1000 == 0:
                print('%d/%d %.2fs'%(i, len(fea_train), time.time() - start))
            fea = sess.run(net, feed_dict={image_path:train_list[i][0]})
            fea_train[i, :] = fea
            label_train[i] = train_list[i][1]
        print('Feature extraction on validation set...')
        for i in range(len(fea_val)):
            if i%1000 == 0:
                print('%d/%d %.2fs'%(i, len(fea_val), time.time() - start))
            fea = sess.run(net, feed_dict={image_path:val_list[i][0]})
            fea_val[i, :] = fea
            label_val[i] = val_list[i][1]

model_name = checkpoints_path.split('/')[-1].split('.ckpt')[0]
if not os.path.exists(os.path.join('./feature', model_name)):
    os.makedirs(os.path.join('./feature', model_name))

save_dir = os.path.join('./feature', model_name, dataset)
np.save(os.path.join(save_dir + '_feature_train.npy'), fea_train)
np.save(os.path.join(save_dir + '_label_train.npy'), label_train)
np.save(os.path.join(save_dir + '_feature_val.npy'), fea_val)
np.save(os.path.join(save_dir + '_label_val.npy'), label_val)


##Regression

In [ ]:
import os
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import LogisticRegression
with open('/content/drive/MyDrive/Cours_MVA/object_recognition/Kaggle_TP3/config.json') as json_data_file:
    config = json.load(json_data_file)

dictionnaire_correspondance={4:0, 9:1,10:2,11:3,12:4,13:5,14:6,15:7,16:8,19:9,20:10,21:11,23:12,26:13,28:14,29:15,
                             30:16,31:17,33:18,34:19}

print("Start load data")
features_train = np.load(os.path.join(config["load_dir"]+ '/_feature_train_assigment.npy'))
labels_train = np.load(os.path.join(config["load_dir"]+ '/_label_train_assigment.npy'))
features_val = np.load(os.path.join(config["load_dir"]+ '/_feature_val_assigment.npy'))
features_test = np.load(os.path.join(config["load_dir"]+ '/_feature_test_assigment.npy'))
labels_val = np.load(os.path.join(config["load_dir"]+ '/_label_val_assigment.npy'))
features_train_crop = np.load(os.path.join(config["load_dir"]+ '/_feature_train_assigment_crop.npy'))
features_val_crop = np.load(os.path.join(config["load_dir"]+ '/_feature_val_assigment_crop.npy'))
features_test_crop = np.load(os.path.join(config["load_dir"]+ '/_feature_test_assigment_crop.npy'))


if config["concatenate"]:
    features_train = np.concatenate((features_train, features_train_crop), axis=1)
    features_val = np.concatenate((features_val, features_val_crop), axis=1)
    features_test = np.concatenate((features_test, features_test_crop), axis=1)


print("End load data")
print("Start regression")
LR = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=100)
LR.fit(features_train, labels_train)
print("End regression")

labels_pred = LR.predict(features_val)
num_class = 20
acc = np.zeros((num_class, num_class), dtype=np.float32)
for i in range(len(labels_val)):
    acc[dictionnaire_correspondance[labels_val[i]], dictionnaire_correspondance[labels_pred[i]]] += 1.0

print('Accuracy on Validation: %f' % (sum([acc[i,i] for i in range(num_class)]) / len(labels_val)))

print("Start Predict and create file csv")
labels_pred = LR.predict(features_test)
df = pd.read_csv('kaggle_template.csv')
for dossier, sous_dossiers, fichiers in os.walk('bird_dataset/test_images/mistery_category'):
    for num, fichier in enumerate(fichiers):
        num_photo = df.loc[df['Id'] == fichier.split('.')[0]].index[0]
        df.Category[num_photo] = dictionnaire_correspondance[int(labels_pred[num])]
print("Succesfully wrote kaggle.csv, you can upload this file to the kaggle competition website")

df.to_csv('kaggle.csv',index=False)